In [8]:
import numpy as np
import cv2
import glob

# termination criteria
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)

# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
objp = np.zeros((5*5,3), np.float32)
objp[:,:2] = np.mgrid[0:5,0:5].T.reshape(-1,2)

# Arrays to store object points and image points from all the images.
objpoints = [] # 3d point in real world space
imgpoints = [] # 2d points in image plane.

images = glob.glob('cv/*.jpg')


In [11]:
for fname in images:
    img = cv2.imread(fname)
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

    # Find the chess board corners
    ret, corners = cv2.findChessboardCorners(gray, (5,5),None)

    # If found, add object points, image points (after refining them)
    if ret == True:
        objpoints.append(objp)

        corners2 = cv2.cornerSubPix(gray,corners,(11,11),(-1,-1),criteria)
        imgpoints.append(corners2)

        # Draw and display the corners
        img = cv2.drawChessboardCorners(img, (5,5), corners2,ret)
        cv2.imshow('img',img)
        cv2.waitKey(500)

cv2.destroyAllWindows()

## Calibration
So now we have our object points and image points we are ready to go for calibration. For that we use the function, cv2.calibrateCamera(). It returns the camera matrix, distortion coefficients, rotation and translation vectors etc.

In [12]:
ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1],None,None)


## Undistortion
We have got what we were trying. Now we can take an image and undistort it. OpenCV comes with two methods, we will see both. But before that, we can refine the camera matrix based on a free scaling parameter using cv2.getOptimalNewCameraMatrix(). If the scaling parameter alpha=0, it returns undistorted image with minimum unwanted pixels. So it may even remove some pixels at image corners. If alpha=1, all pixels are retained with some extra black images. It also returns an image ROI which can be used to crop the result.

So we take a new image (left12.jpg in this case. That is the first image in this chapter)

In [4]:
img = cv2.imread('cv/test1.jpg')
h,  w = img.shape[:2]
newcameramtx, roi=cv2.getOptimalNewCameraMatrix(mtx,dist,(w,h),1,(w,h))

In [5]:
# undistort
dst = cv2.undistort(img, mtx, dist, None, newcameramtx)

# crop the image
x,y,w,h = roi
dst = dst[y:y+h, x:x+w]
cv2.imwrite('cv/calibresult.png',dst)

True

In [6]:
# undistort
mapx,mapy = cv2.initUndistortRectifyMap(mtx,dist,None,newcameramtx,(w,h),5)
dst = cv2.remap(img,mapx,mapy,cv2.INTER_LINEAR)

# crop the image
x,y,w,h = roi
dst = dst[y:y+h, x:x+w]
cv2.imwrite('calibresult.png',dst)

True

In [7]:
mapx.shape

(867, 636)

## Re-projection Error
Re-projection error gives a good estimation of just how exact is the found parameters. This should be as close to zero as possible. Given the intrinsic, distortion, rotation and translation matrices, we first transform the object point to image point using cv2.projectPoints(). Then we calculate the absolute norm between what we got with our transformation and the corner finding algorithm. To find the average error we calculate the arithmetical mean of the errors calculate for all the calibration images.

In [8]:
tot_error = 0
for i in range(len(objpoints)):
    imgpoints2, _ = cv2.projectPoints(objpoints[i], rvecs[i], tvecs[i], mtx, dist)
    error = cv2.norm(imgpoints[i],imgpoints2, cv2.NORM_L2)/len(imgpoints2)
    tot_error += error

print("total error: ", tot_error/len(objpoints))

total error:  0.25709820340049283


## Pose Estimation

In [14]:
def draw(img, corners, imgpts):
    corner = tuple(corners[0].ravel())
    img = cv2.line(img, corner, tuple(imgpts[0].ravel()), (255,0,0), 5)
    img = cv2.line(img, corner, tuple(imgpts[1].ravel()), (0,255,0), 5)
    img = cv2.line(img, corner, tuple(imgpts[2].ravel()), (0,0,255), 5)
    return img

In [9]:
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)
objp = np.zeros((5*5,3), np.float32)
objp[:,:2] = np.mgrid[0:5,0:5].T.reshape(-1,2)

axis = np.float32([[3,0,0], [0,3,0], [0,0,-3]]).reshape(-1,3)

Now, as usual, we load each image. Search for 7x6 grid. If found, we refine it with subcorner pixels. Then to calculate the rotation and translation, we use the function, cv2.solvePnPRansac(). Once we those transformation matrices, we use them to project our axis points to the image plane. In simple words, we find the points on image plane corresponding to each of (3,0,0),(0,3,0),(0,0,3) in 3D space. Once we get them, we draw lines from the first corner to each of these points using our draw() function. Done !!!

In [3]:
images = glob.glob('cv/c*.jpg')

In [15]:
for fname in images:
    img = cv2.imread(fname)
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    ret, corners = cv2.findChessboardCorners(gray, (5,5),None)

    if ret == True:
        corners2 = cv2.cornerSubPix(gray,corners,(11,11),(-1,-1),criteria)

        # Find the rotation and translation vectors.
        _, rvecs, tvecs, inliers = cv2.solvePnPRansac(objp, corners2, mtx, dist)

        # project 3D points to image plane
        imgpts, jac = cv2.projectPoints(axis, rvecs, tvecs, mtx, dist)

        img = draw(img,corners2,imgpts)
        cv2.imshow('img',img)
        k = cv2.waitKey(0) & 0xff
        if k == 's':
            cv2.imwrite(fname[:6]+'.png', img)

cv2.destroyAllWindows()

In [19]:
corners2.shape

(25, 1, 2)

In [20]:
objp.shape

(25, 3)

In [21]:
_, rvecs, tvecs, inliers = cv2.solvePnPRansac(objp, corners2, mtx, dist)

### Render a Cube
If you want to draw a cube, modify the draw() function and axis points as follows.

Modified draw() function:

In [16]:
def draw(img, corners, imgpts):
    imgpts = np.int32(imgpts).reshape(-1,2)

    # draw ground floor in green
    img = cv2.drawContours(img, [imgpts[:4]],-1,(0,255,0),-3)

    # draw pillars in blue color
    for i,j in zip(range(4),range(4,8)):
        img = cv2.line(img, tuple(imgpts[i]), tuple(imgpts[j]),(255),3)

    # draw top layer in red color
    img = cv2.drawContours(img, [imgpts[4:]],-1,(0,0,255),3)

    return img

In [15]:
axis = np.float32([[0,0,0], [0,3,0], [3,3,0], [3,0,0],
                   [0,0,-3],[0,3,-3],[3,3,-3],[3,0,-3] ])

## Try to get camera pose

In [39]:
def trans_matrix(r,t):
    # T (ki to t), rotate and transport matrix
    (a,b,c) = (r[0][0], r[1][0], r[2][0])
    (x,y,z) = (t[0][0],t[1][0],t[2][0])
    Rz = np.matrix([[1,0,0],
          [0, np.cos(a), -np.sin(a)],
         [0, np.sin(a), np.cos(a)]])
    Ry = np.matrix([[np.cos(b), 0 , np.sin(b)],
          [0 , 1, 0],
          [-np.sin(b), 0 , np.cos(b)]])
    Rx = np.matrix([[np.cos(c), -np.sin(c), 0],
         [np.sin(c), np.cos(c), 0],
         [0,0,1]])
    R = np.dot(Rz,np.dot(Ry,Rx))
    T = np.concatenate((R, np.matrix([[x],[y],[z]])), axis=1)
    
    return T

In [33]:
def pi_trans(x,l):
    u = np.zeros((l,2))
    for i in range(l):
        u[i,0] = x[0,i]/x[2,i]
        u[i,1] = x[1,i]/x[2,i]
    return u

In [34]:
rvecs_set = []
tvecs_set = []

fname = 'cv/test1.jpg'
img = cv2.imread(fname)
gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
ret, corners = cv2.findChessboardCorners(gray, (5,5),None)

if ret == True:
    corners2 = cv2.cornerSubPix(gray,corners,(11,11),(-1,-1),criteria)

    # Find the rotation and translation vectors.
    _, rvecs, tvecs, inliers = cv2.solvePnPRansac(objp, corners2, mtx, dist)

    # project 3D points to image plane
    imgpts, jac = cv2.projectPoints(axis, rvecs, tvecs, mtx, dist)
        
    rvecs_set.append(rvecs)
    tvecs_set.append(tvecs)

In [35]:
axis

array([[ 0.,  0.,  0.],
       [ 0.,  3.,  0.],
       [ 3.,  3.,  0.],
       [ 3.,  0.,  0.],
       [ 0.,  0., -3.],
       [ 0.,  3., -3.],
       [ 3.,  3., -3.],
       [ 3.,  0., -3.]], dtype=float32)

### transport matrix

transposr from axis_cube to the cube in the image

T_origin_image

In [36]:
T = trans_matrix(rvecs_set[0], tvecs_set[0])
T

matrix([[ 9.85295221e-01,  1.20368386e-01, -1.21263263e-01,
         -1.12759475e+00],
        [-1.70786660e-01,  7.14693866e-01, -6.78265873e-01,
         -3.48967636e+00],
        [ 5.02434150e-03,  6.89002271e-01,  7.24741765e-01,
          1.09166041e+01]])

In [37]:
cube = np.concatenate((axis.T, [[1,1,1,1,1,1,1,1]]), axis=0)
temp = np.dot(mtx, np.dot(T, cube))

In [54]:
T1 = np.concatenate((T, np.array([[0,0,0,1]])), axis=0)
T0 = np.linalg.inv(T1)
np.dot(T0,np.dot(T1,cube))

matrix([[-3.33066907e-16, -1.66533454e-16,  3.00000000e+00,
          3.00000000e+00, -2.22044605e-16,  0.00000000e+00,
          3.00000000e+00,  3.00000000e+00],
        [ 8.88178420e-16,  3.00000000e+00,  3.00000000e+00,
          0.00000000e+00,  8.88178420e-16,  3.00000000e+00,
          3.00000000e+00,  0.00000000e+00],
        [ 1.77635684e-15,  1.77635684e-15,  1.77635684e-15,
          0.00000000e+00, -3.00000000e+00, -3.00000000e+00,
         -3.00000000e+00, -3.00000000e+00],
        [ 1.00000000e+00,  1.00000000e+00,  1.00000000e+00,
          1.00000000e+00,  1.00000000e+00,  1.00000000e+00,
          1.00000000e+00,  1.00000000e+00]])

In [38]:
cube = np.concatenate((axis.T, [[1,1,1,1,1,1,1,1]]), axis=0)
temp = np.dot(mtx, np.dot(T, cube))
temp = pi_trans(temp,8)
print('temp:',temp)
print('imgpts:', imgpts)

temp: [[322.93538373 196.69689097]
 [359.55767556 374.21029025]
 [547.78684345 341.92022197]
 [546.80619917 158.54604515]
 [336.11217898 322.62418871]
 [377.58105411 511.76281309]
 [603.59326194 472.79534702]
 [615.54104805 274.78483516]]
imgpts: [[[326.26077 195.00955]]

 [[183.65842 361.10806]]

 [[338.9686  500.62103]]

 [[485.4027  350.217  ]]

 [[303.63016 179.38838]]

 [[117.29534 401.15668]]

 [[324.29642 582.23126]]

 [[518.2971  387.90305]]]
